In [1]:
import torch
import numpy as np

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
target = np.array([[[1, 0, 1], [1, 1, 1], [0, 0, 1]]])
target = torch.from_numpy(target).type(torch.float32)

output = torch.tensor([[[[0, 0, 0], [0, 0, 1], [1, 1, 1]], [[1, 1, 1], [1, 1, 0], [0, 0, 0]]]], dtype = torch.float32)

print(target.size(), target.dtype, output.size(), output.dtype)

selective = 0
n_cls = 2

NCHW_tonumpy = lambda x : x.to('cpu').detach().numpy().transpose(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
NHW_tonumpy = lambda x: x.to('cpu').detach().numpy()

target = NHW_tonumpy(target).astype('uint8')

output = NCHW_tonumpy(output)
pred = np.argmax(output, axis = -1).astype('uint8')

print(target.shape, target.dtype, pred.shape, pred.dtype)

torch.Size([1, 3, 3]) torch.float32 torch.Size([1, 2, 3, 3]) torch.float32
(1, 3, 3) uint8 (1, 3, 3) uint8


In [10]:
target, pred

(array([[[1, 0, 1],
         [1, 1, 1],
         [0, 0, 1]]], dtype=uint8),
 array([[[1, 1, 1],
         [1, 1, 0],
         [0, 0, 0]]], dtype=uint8))

In [11]:
from compute_metric import Evaluator

evaluator = Evaluator(num_class=n_cls, selective=selective)

evaluator.add_batch(label=target, pred=pred)

In [12]:
evaluator.Confusion_Matrix()

[[2. 1.]
 [2. 4.]]


array([[2., 1.],
       [2., 4.]])

In [13]:
Acc = evaluator.get_Pixel_Accuracy()
Acc_class = evaluator.get_Pixel_Accuracy_Class()
Prec = evaluator.get_Precision()
Recall = evaluator.get_Recall()
F1_Score = evaluator.get_F1_Score(Prec, Recall)
mIoU = evaluator.get_mIoU()
IoU_class = evaluator.get_IoU_Class()
# FWIoU = evaluator.get_FWIoU()

print(f'    Acc:{Acc}')
print(f'    Acc_class:{Acc_class}')
print(f'    Prec:{Prec}, Recall:{Recall}, F1_Score:{F1_Score}')
print(f'    mIoU:{mIoU}')
print(f'    IoU_class:{IoU_class}')

    Acc:0.6666666666666666
    Acc_class:0.6666666666666666
    Prec:[0.5 0.8], Recall:[0.66666667 0.66666667], F1_Score:[0.57142857 0.72727273]
    mIoU:0.4857142857142857
    IoU_class:[0.4        0.57142857]


In [14]:
fn_sigmoid = lambda x : 1/(1+ np.exp(-x))

fn_sigmoid(0.5)

0.6224593312018546

In [15]:
fn_sigmoid(0)

0.5